In [1]:
import sys
import numpy as np

In [2]:
automato_nome = input('Digite o nome do arquivo:')

Digite o nome do arquivo: automato.txt


Tomando o seguinte tipo de arquivo:<br>
{Estados} <br>
{Alfabeto} <br>
{Simbolos da Pilha} <br>
{Estado Inicial} <br>
{Sibolo de Inicio de Pilha} <br>
{Estado Final} <br>
Função de Transição: <br>
Estado  Simbolo_Lido Estado_Pilha Transicao





In [3]:
file = open(automato_nome, "r")
linhas = file.readlines()

### Lendo estados

In [4]:
estados = linhas[0].split(",")
estados[0] = estados[0][1:]
aux = len(estados)-1
estados[aux]= estados[aux][0:-2]
print(estados)

['q0', 'q1', 'q2']


### Lendo Alfabeto

In [5]:
alfabeto = linhas[1].split(',')
alfabeto[0] = alfabeto[0][1:]
aux = len(alfabeto)-1
alfabeto[aux]= alfabeto[aux][0:-2]
alfabeto.append("E")
if '' in alfabeto: alfabeto.remove('')
alfabeto

['0', '1', 'E']

### Lendo Símbolos da Pilha

In [6]:
simbolos_pilha = linhas[2].split(',')
simbolos_pilha[0] = simbolos_pilha[0][1:]
aux = len(simbolos_pilha)-1
simbolos_pilha[aux] = simbolos_pilha[aux][0:-2]
simbolos_pilha

['0', '1', 'Z']

### Lendo Estado final, Estado inicial e Estado Inicial da Pilha

In [7]:
estado_inicial = linhas[3][1:-2]
simbolo_pilha_inicial = linhas[4][1:-2]
estados_finais = []
estados_finais = linhas[5].split(",")
estados_finais[0] = estados_finais[0][1:]
estados_finais[-1]= estados_finais[0][0:-2]
if '' in estados_finais: estados_finais.remove('')
    
print('Estado Inicial: {0}\nEstados Finais: {1}\nSimbolo Inicial da Pilha: {2}'.format(estado_inicial,estados_finais,simbolo_pilha_inicial))

Estado Inicial: q0
Estados Finais: ['q2']
Simbolo Inicial da Pilha: Z


### Matriz de transição
Matriz de transição da seguinte forma: [estado][simbolo][pilha] -> retorna um conjunto de listas (transicao, att_pilha)<br>
att_pilha = [codigo, simbolo] <br>
Códigos:
    * 0 = desempilha
    * 1 = não modifica
    * 2 = empilha simbolo

In [8]:
#Descartando linhas ja lidas
linhas  = linhas[7:]

In [9]:

matriz_transicoes = []
for i in range(len(estados)):
    matriz_transicoes.insert(i, [])
    for j in range(len(alfabeto)):
        matriz_transicoes[i].insert(j, [])
        for k in range(len(simbolos_pilha)):
            matriz_transicoes[i][j].insert(k, [])
            
for line in linhas:
    l = line.split()
    est = l[0];
    sim = l[1]
    pil = l[2]
    l[3] = l[3][1:-1]
    trans = []
    #Definindo as tuplas de transicao
    for t in l[3].split(';'):
        t_ = t[1:-1].split(',')
        t_pilha = None
        if 'E' == t_[1]:
            t_pilha = ([0])
        elif t_[1] in simbolos_pilha:
            t_pilha = ([1])
        else:
            t_pilha= ([2, simbolos_pilha.index(t_[1][0])])
            
        trans.append([estados.index(t_[0]), t_pilha])
    print("[{0} | {1} | {2} | {3}]".format(est, sim, pil, trans))
    
    matriz_transicoes[estados.index(est)][alfabeto.index(sim)][simbolos_pilha.index(pil)] = trans
matriz_transicoes[0][0][0]

[q0 | 0 | Z | [[0, [2, 0]]]]
[q0 | 1 | Z | [[0, [2, 1]]]]
[q0 | 0 | 0 | [[0, [2, 0]]]]
[q0 | 0 | 1 | [[0, [2, 0]]]]
[q0 | 1 | 0 | [[0, [2, 1]]]]
[q0 | 1 | 1 | [[0, [2, 1]]]]
[q0 | E | Z | [[1, [1]]]]
[q0 | E | 0 | [[1, [1]]]]
[q0 | E | 1 | [[1, [1]]]]
[q1 | 0 | 0 | [[1, [0]]]]
[q1 | 1 | 1 | [[1, [0]]]]
[q1 | E | Z | [[2, [1]]]]


[[0, [2, 0]]]

### Conversão caso o autômato for de pilha vazia

In [10]:
if(len(estados_finais) == 0):
    #Adicionando um novo simbolo para representar a pilha vazia
    simbolos_pilha.append('X')
    
    #Adicionando um novo estado inicial e um estado final
    estados_finais.append('pf')
    estados.append('p0')
    estados.append('pf')
    
    #Adicionando transicao inicial na matriz de transições
    ini_pos = estados.index('p0')
    fim_pos = estados.index('pf')
    pos_pilha = simbolos_pilha.index('X')
    matriz_transicoes.insert(ini_pos, [])
    #Adicionando nova linha na matriz de transições
    matriz_transicoes.insert(ini_pos, [[[] for j in range(len(simbolos_pilha))] for i in range(len(alfabeto))])
    
    trans_inicial = [[estados.index(estado_inicial), [2, simbolos_pilha.index(simbolo_pilha_inicial)]]]
    matriz_transicoes[ini_pos][alfabeto.index('E')][pos_pilha] = trans_inicial
    matriz_transicoes.insert(fim_pos, [[[] for j in range(len(simbolos_pilha))] for i in range(len(alfabeto))])
    
    for i in range(len(estados)-2):
        matriz_transicoes[i][alfabeto.index('E')].insert(simbolos_pilha.index('X'), [])
        matriz_transicoes[i][alfabeto.index('E')][simbolos_pilha.index('X')] =[[estados.index(estados_finais[0]), [1]]]
    
    for i in range(len(estados)-2):
        for j in range(len(alfabeto)):
            if j !=  alfabeto.index('E'):
               matriz_transicoes[i][j].append([[]*len(simbolos_pilha)])
            
    estado_inicial = 'p0'
    simbolo_pilha_inicial = 'X'

In [ ]:
def leia(v, estad, pilha):
    if len(v) > 0:
        simb_pos = alfabeto.index(v[0])
        pil_pos = pilha.pop()
        p_estados = matriz_transicoes[estad][simb_pos][pil_pos] # retorna: lista de [estado, [op_pilha, simb_pilha]]
        
#        print("::::::Normais:::::::\nPalavra: {2}\nSimbolo: {3}, Sim Pilha:{4}\nEstados: {0}\nPilha: {1}\n------------------------".format(p_estados, pilha, v, v[0], pil_pos))
        for tupla in p_estados: # tupla tem o formato [estado, [op_pilha, pilha_simbolo]]
#            print('Testando estado:{0}\tSimbolo:{1}'.format(tupla,pil_pos))
            if tupla == [] :
                continue
            n_est = tupla[0]
            op_pilha = tupla[1]
            pilha_ = pilha.copy()
            if op_pilha[0] == 1:
                pilha_.append(pil_pos)
            elif op_pilha[0] == 2:
                pilha_.append(pil_pos)
                pilha_.append(op_pilha[1])
            
            #Com a operação ja feita na pilha, siga
            b = leia(v[1:], n_est, pilha_)
            if b: return True
            
        
#       print(':::::::vazios::::::')
        p_estados_vazios = matriz_transicoes[estad][alfabeto.index('E')][pil_pos]
#        print("Palavra: {2}\nSimbolo: {3}, Sim Pilha:{4}\nEstados: {0}\nPilha: {1}\n--------------------------".format(p_estados_vazios, pilha, v, v[0], pil_pos))
     
     
        for tupla in p_estados_vazios: # tupla tem o formato [estado, [op_pilha, pilha_simbolo]]
#            print('Testando estado:{0}\tSimbolo:{1}'.format(tupla,pil_pos))
            if tupla == [] :
                continue
            n_est = tupla[0]
            pilha_ = pilha.copy()
            op_pilha = tupla[1]
            if op_pilha[0] == 1:
                pilha_.append(pil_pos)
            elif op_pilha[0] == 2:
                pilha_.append(pil_pos)
                pilha_.append(op_pilha[1])
            
            #Com a operação ja feita na pilha, siga
            b = leia(v, n_est, pilha_)
            if b: return True

        return False
    else:
        pilha_ = pilha.copy()
        fecho = e_fecho(estad, pilha_)
#        print(fecho)
        if any(ef in list(map(lambda a: estados.index(a),estados_finais)) for ef in fecho):
#            print('::::::::::::Final:::::::::')
            return True
        else: return False

In [ ]:
def e_fecho(estado, pilha):
        visitados = []
        lista = []
        e_fechoh(estado, lista, pilha, visitados)
        return visitados

def e_fechoh(estado, lista, pilha, visitados):
        pil = pilha.pop()
        p_estados = matriz_transicoes[estado][alfabeto.index('E')][pil]
#        print('E-fecho:{0}'.format(p_estados))
        visitados.append(estado)
          #verificar se podemos usar transições vazias para chegar a um estado final, lendo o ultimo simbolo*
        for tupla in p_estados:
 #           print('Calculando e-fecho:{0}\tSimbolo:{1}'.format(tupla,pil))
            if tupla == [] :
                continue
            n_est = tupla[0]
            pilha_ = pilha.copy()
            op_pilha = tupla[1]
            if op_pilha[0] == 1:
                pilha_.append(pil)
            elif op_pilha[0] == 2:
                pilha_.append(pil)
                pilha_.append(op_pilha[1])
            if  n_est not in lista:
                #lista.extend(matriz_transicoes[p][alfabeto.index('E')])
                lista.append(estado)
                visitados.append(n_est)
                if len(pilha_) > 0:
                    e_fechoh(n_est, lista, pilha_, visitados)

### * Como o e-fecho inclui o próprio estado atual, considera-se apenas o vetor de e-fechos

In [ ]:
while True:
    palavra = input("Digite uma palavra: ")
    pode_processar = True
    for a in palavra:
      if a not in alfabeto:
        pode_processar = False
        break

    if pode_processar:
        pilha = [simbolos_pilha.index(simbolo_pilha_inicial)]
        if(leia(palavra, estados.index(estado_inicial), pilha)): print("A palavra {} eh aceita".format(palavra))
        else: print("A palavra {} não eh aceita".format(palavra))
    else: print("A palavra {} não pode ser processada".format(palavra))

Digite uma palavra:  110011


[0, 1, 1]
[1]
[1, 2, 2]
A palavra 110011 eh aceita


In [ ]:
# print(matriz_transicoes[estados.index('q1')][alfabeto.index('E')][simbolos_pilha.index('X')])
# print(simbolos_pilha)
# print(matriz_transicoes[1][alfabeto.index('E')][2])